In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

C:\Users\alexanjs\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pd.read_csv('./data/raw_results.csv')
df.head()

,Date,Round,Region,Winning Seed,Winner,Winning Score,Losing Seed,Loser,Losing Score,Overtime
0,3/14/85,Round of 64,East,1,Georgetown,68,16,Lehigh,43,NaN
1,3/14/85,Round of 64,East,4,"Loyola, Illinois",59,13,Iona,58,NaN
2,3/14/85,Round of 64,East,5,Southern Methodist,85,12,Old Dominion,68,NaN
3,3/14/85,Round of 64,East,8,Temple,60,9,Virginia Tech,57,NaN
4,3/14/85,Round of 64,Midwest,1,Oklahoma,96,16,North Carolina A&T,83,NaN


In [3]:

df.dtypes

Date             object
Round            object
Region           object
Winning Seed      int64
Winner           object
Winning Score     int64
Losing Seed       int64
Loser            object
Losing Score      int64
Overtime         object
dtype: object

In [4]:
df['Year'] = df['Date'].astype(str).str[-2:]

In [5]:
df.head()

,Date,Round,Region,Winning Seed,Winner,Winning Score,Losing Seed,Loser,Losing Score,Overtime,Year
0,3/14/85,Round of 64,East,1,Georgetown,68,16,Lehigh,43,NaN,85
1,3/14/85,Round of 64,East,4,"Loyola, Illinois",59,13,Iona,58,NaN,85
2,3/14/85,Round of 64,East,5,Southern Methodist,85,12,Old Dominion,68,NaN,85
3,3/14/85,Round of 64,East,8,Temple,60,9,Virginia Tech,57,NaN,85
4,3/14/85,Round of 64,Midwest,1,Oklahoma,96,16,North Carolina A&T,83,NaN,85


In [6]:
dictionary = {'National Championship': 0, 
              'National Semifinals': 1, 
              'Elite Eight': 2, 
              'Sweet Sixteen': 3, 
              'Round of 32': 4, 
              'Round of 64': 5, 
              'Opening Round': 6}

In [7]:
df['RoundNumber'] = df.Round.map(lambda x: dictionary[x])

In [8]:
df.tail()

,Date,Round,Region,Winning Seed,Winner,Winning Score,Losing Seed,Loser,Losing Score,Overtime,Year,RoundNumber
2045,3/27/16,Elite Eight,East,1,North Carolina,88,6,Notre Dame,74,NaN,16,2
2046,3/27/16,Elite Eight,Midwest,10,Syracuse,68,1,Virginia,62,NaN,16,2
2047,4/2/16,National Semifinals,NaN,1,North Carolina,83,10,Syracuse,66,NaN,16,1
2048,4/2/16,National Semifinals,NaN,2,Villanova,95,2,Oklahoma,51,NaN,16,1
2049,4/4/16,National Championship,NaN,2,Villanova,77,1,North Carolina,74,NaN,16,0


In [9]:
df_new = df[['Winner', 'Loser', 'Year', 'RoundNumber']]

In [10]:
df_new.head()

,Winner,Loser,Year,RoundNumber
0,Georgetown,Lehigh,85,5
1,"Loyola, Illinois",Iona,85,5
2,Southern Methodist,Old Dominion,85,5
3,Temple,Virginia Tech,85,5
4,Oklahoma,North Carolina A&T,85,5


In [11]:
list_of_losers = []

for index, row in df_new.iterrows():
    loser = row.Loser
    rd = row.RoundNumber
    year = row.Year
    list_of_losers.append({'School': loser, 'Round': rd, 'Year': year})
    if rd is 0: 
        list_of_losers.append({'School': row.Winner, 'Round': rd-1, 'Year': year})
    
targets = pd.DataFrame(list_of_losers)
targets['Round'] = targets['Round'] + 1
targets.head()
targets['School'] = targets['School'].astype(str)
targets['Year'] = targets['Year'].astype('int64')
targets.dtypes

Round      int64
School    object
Year       int64
dtype: object

In [12]:
targets.to_csv('./data/round_team_year.csv')

In [13]:
stats = pd.DataFrame()

for i in range(16):
    temp = pd.read_excel('./data/'+str(2000 + i)+'-'+str(2001 + i)+'.xlsx', skiprows=1)
    temp['Year'] = i
    stats = stats.append(temp, ignore_index=True)

stats.shape

(5354, 35)

In [14]:
stats.tail()

,Rk,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,...,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Year
5349,347,Wright State,35,22,13,0.629,0.18,-1.76,13.0,5.0,...,621,0.694,232.0,1094,478,193,75,397.0,770.0,15
5350,348,Wyoming,32,14,18,0.438,0.09,2.19,7.0,11.0,...,747,0.716,181.0,989,392,132,84,358.0,594.0,15
5351,349,Xavier NCAA,34,28,6,0.824,18.58,8.43,14.0,4.0,...,857,0.730,436.0,1387,548,246,101,429.0,678.0,15
5352,350,Yale NCAA,30,23,7,0.767,9.08,-1.03,13.0,1.0,...,699,0.670,407.0,1208,452,170,137,398.0,530.0,15
5353,351,Youngstown State,32,11,21,0.344,-10.86,-0.86,6.0,12.0,...,619,0.711,283.0,1081,467,219,80,387.0,571.0,15


In [15]:
stats=stats.drop(['Rk', 'Unnamed: 16'], axis=1)

In [16]:
stats.columns

Index(['School', 'G', 'W', 'L', 'W-L%', 'SRS', 'SOS', 'W.1', 'L.1', 'W.2',
       'L.2', 'W.3', 'L.3', 'Tm.', 'Opp.', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'Year'],
      dtype='object')

In [17]:
stats['School'] = stats['School'].str.replace('NCAA', '')
stats['School'] = stats['School'].str.strip()

In [18]:
table = pd.merge(stats, targets, on=['School', 'Year'])

In [19]:
table[table['School'] == 'Duke']

,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,W.2,...,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Year,Round
12,Duke,39,35,4,0.897,32.18,11.98,13.0,3.0,13,...,0.696,538.0,1505,701,411,196,531.0,659.0,0,4
69,Duke,39,35,4,0.897,32.18,11.98,13.0,3.0,13,...,0.696,538.0,1505,701,411,196,531.0,659.0,1,0
129,Duke,33,26,7,0.788,19.86,8.38,11.0,5.0,15,...,0.712,431.0,1206,444,286,178,462.0,698.0,2,4
192,Duke,37,31,6,0.838,25.67,10.86,13.0,3.0,14,...,0.736,474.0,1351,536,340,240,511.0,699.0,3,4
252,Duke,33,27,6,0.818,23.01,10.04,11.0,5.0,14,...,0.713,432.0,1261,421,287,220,454.0,612.0,4,2
311,Duke,36,32,4,0.889,21.77,8.77,14.0,2.0,14,...,0.761,350.0,1204,546,337,202,496.0,621.0,5,4
370,Duke,33,22,11,0.667,18.85,10.49,8.0,8.0,15,...,0.689,379.0,1165,433,241,157,509.0,577.0,6,4
428,Duke,34,28,6,0.824,21.52,9.28,13.0,3.0,15,...,0.697,426.0,1255,497,295,133,462.0,654.0,7,6
496,Duke,37,30,7,0.811,20.10,9.07,11.0,5.0,16,...,0.728,490.0,1348,492,311,146,454.0,671.0,8,5
553,Duke,40,35,5,0.875,25.21,9.21,13.0,3.0,17,...,0.759,569.0,1560,555,266,162,429.0,722.0,9,4


In [20]:
table.shape

(965, 34)

In [21]:
table.head()

,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,W.2,...,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Year,Round
0,Appalachian State,31,11,20,0.355,-12.13,-4.24,7.0,9.0,6,...,0.675,410.0,1172,444,193,79,547.0,598.0,0,6
1,Arizona,36,28,8,0.778,26.84,12.56,15.0,3.0,13,...,0.748,487.0,1433,585,283,159,546.0,630.0,0,5
2,Arkansas,31,20,11,0.645,16.61,6.16,10.0,6.0,16,...,0.690,410.0,1100,476,365,124,451.0,654.0,0,6
3,Auburn,32,18,14,0.563,11.48,7.14,7.0,9.0,14,...,0.651,466.0,1243,495,265,155,450.0,629.0,0,5
4,Ball State,30,18,12,0.600,3.12,0.30,11.0,7.0,9,...,0.652,412.0,1152,381,153,98,402.0,527.0,0,6


In [22]:
table = table.fillna(0)

In [23]:
table.to_csv('./data/dataset.csv')
print(table.shape)

(965, 34)


In [24]:
table.Round.value_counts()


6    460
5    239
4    119
3     61
2     32
7     22
1     16
0     16
Name: Round, dtype: int64

In [25]:
table.shape

(965, 34)

In [26]:
460/16

28.75

In [27]:
table.shape

(965, 34)

In [32]:
temp = pd.DataFrame()
temp = temp.append(table[table['Round'] == 0].sample(frac=16/len(table[table['Round'] == 0])), ignore_index=True)
temp = temp.append(table[table['Round'] == 1].sample(frac=16/len(table[table['Round'] == 1])), ignore_index=True)
temp = temp.append(table[table['Round'] == 2].sample(frac=16/len(table[table['Round'] == 2])), ignore_index=True)
temp = temp.append(table[table['Round'] == 3].sample(frac=16/len(table[table['Round'] == 3])), ignore_index=True)
temp = temp.append(table[table['Round'] == 4].sample(frac=16/len(table[table['Round'] == 4])), ignore_index=True)
temp = temp.append(table[table['Round'] == 5].sample(frac=16/len(table[table['Round'] == 5])), ignore_index=True)
temp = temp.append(table[table['Round'] == 6].sample(frac=16/len(table[table['Round'] == 6])), ignore_index=True)

# for i in range(29):
#     temp = temp.append(table[table['Round'] == 0], ignore_index=True)
# for i in range(29):
#     temp = temp.append(table[table['Round'] == 1], ignore_index=True)
# for i in range(15):
#     temp = temp.append(table[table['Round'] == 2], ignore_index=True)
# for i in range(7):
#     temp = temp.append(table[table['Round'] == 3], ignore_index=True)
# for i in range(4):
#     temp = temp.append(table[table['Round'] == 4], ignore_index=True)
# for i in range(2):
#     temp = temp.append(table[table['Round'] == 5], ignore_index=True)
# temp = temp.append(table[table['Round'] == 6], ignore_index=True)

print(temp.shape)
temp

(112, 34)


,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,W.2,...,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Year,Round
0,Duke,39,35,4,0.897,32.18,11.98,13.0,3.0,13,...,0.696,538.0,1505,701,411,196,531.0,659.0,1,0
1,Connecticut,31,23,8,0.742,18.82,7.62,13.0,3.0,14,...,0.700,478.0,1412,531,189,275,468.0,492.0,4,0
2,Duke,37,32,5,0.865,24.51,8.32,13.0,3.0,17,...,0.753,446.0,1399,529,269,162,447.0,657.0,10,0
3,Michigan State,33,28,5,0.848,25.12,9.54,13.0,3.0,17,...,0.719,514.0,1404,579,210,133,451.0,611.0,0,0
4,Florida,40,35,5,0.875,23.82,7.34,13.0,3.0,18,...,0.690,447.0,1504,614,266,194,565.0,626.0,6,0
5,Kentucky,33,21,12,0.636,13.94,5.78,12.0,6.0,16,...,0.642,379.0,1163,441,185,217,430.0,566.0,12,0
6,Connecticut,34,20,14,0.588,12.27,8.07,8.0,10.0,13,...,0.662,415.0,1151,457,199,233,418.0,492.0,11,0
7,North Carolina,37,20,17,0.541,10.14,9.83,5.0,11.0,14,...,0.653,556.0,1552,577,250,210,562.0,564.0,9,0
8,North Carolina,31,23,8,0.742,18.48,7.77,12.0,4.0,14,...,0.708,432.0,1237,552,242,127,512.0,531.0,5,0
9,Syracuse,31,23,8,0.742,13.06,7.61,11.0,5.0,14,...,0.643,423.0,1180,465,235,205,396.0,554.0,3,0


In [35]:
temp.Round.value_counts()

6    16
5    16
4    16
3    16
2    16
1    16
0    16
Name: Round, dtype: int64

In [36]:
temp.head()

,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,W.2,...,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Year,Round
0,Duke,39,35,4,0.897,32.18,11.98,13.0,3.0,13,...,0.696,538.0,1505,701,411,196,531.0,659.0,1,0
1,Connecticut,31,23,8,0.742,18.82,7.62,13.0,3.0,14,...,0.700,478.0,1412,531,189,275,468.0,492.0,4,0
2,Duke,37,32,5,0.865,24.51,8.32,13.0,3.0,17,...,0.753,446.0,1399,529,269,162,447.0,657.0,10,0
3,Michigan State,33,28,5,0.848,25.12,9.54,13.0,3.0,17,...,0.719,514.0,1404,579,210,133,451.0,611.0,0,0
4,Florida,40,35,5,0.875,23.82,7.34,13.0,3.0,18,...,0.690,447.0,1504,614,266,194,565.0,626.0,6,0


In [37]:
table = temp
table.to_csv('./data/dataset.csv')

TODO: 

1. Add all data to the data folder and finalize the data_cleaning script. DONE
1. Use PCA DONE
2. Go through the initial testing of every classifier(regressor as well) in each of the separate documents with all data. DONE
    a. Logistic
    b. KNN
    c. Decision Tree
    d. Random Forest
    e. Boosting Tree
3. Run GridSearchCV in order to find the best one.
3. Ask Shibberu about the regressor/classifier problem where r2 is better on classifier but rmse is higher 
4. Begin analysis on all of this in order to make it the best it can be.
5. Output to file with rankings
6. Write the Paper


In [39]:
temp.columns

Index(['School', 'G', 'W', 'L', 'W-L%', 'SRS', 'SOS', 'W.1', 'L.1', 'W.2',
       'L.2', 'W.3', 'L.3', 'Tm.', 'Opp.', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'Year', 'Round'],
      dtype='object')